In [1]:
import pyodbc
import pandas as pd


cnxn = pyodbc.connect("Driver={SQL Server};"
                    "Server=10.1.252.xx;"
                    "Database=PDATA_DW;"
                    "UID=sql;"
                    "PWD=sqlpassword;")
cursor = cnxn.cursor()


#每月活躍用戶
query2 = "SELECT 登入年月,COUNT(z.MDN) AS 使用門號數\
         FROM(SELECT DISTINCT CONVERT(VARCHAR(6),A.OP_DT,112) AS 登入年月,MDN\
         FROM PDATA_DW.SVIEW.EVN_GTAPP_LOG A\
         WHERE OP_DT between '2021-01-01' and convert(char(10),getdate()-1,120)\
         and A.mdn not in('GUEST'))Z\
         GROUP BY 登入年月\
         ORDER BY 登入年月"
df = pd.read_sql(query2, cnxn)


#每日活躍用戶
query3 = "SELECT date,COUNT(z.MDN) AS 使用門號數\
          FROM(SELECT DISTINCT CONVERT(VARCHAR(10),A.OP_DT,111) AS date,MDN\
          FROM PDATA_DW.SVIEW.EVN_GTAPP_LOG A\
          WHERE OP_DT between '2021-01-01' and convert(char(10),getdate()-1,120)\
          and A.mdn not in('GUEST'))Z\
          GROUP BY date\
          ORDER BY date"
df2 = pd.read_sql(query3, cnxn)


#每月首登用戶
query4 = "SELECT  登入年月,COUNT(z.MDN) AS 使用門號數\
       FROM(SELECT  CONVERT(VARCHAR(6),A.FIRST_LGN_DT,112) AS 登入年月,MDN\
       FROM [PDATA_DW].[SVIEW].[EVN_IVR_TB_APP_MDN_FIRST] A\
       WHERE FIRST_LGN_DT between '2021-01-01' and convert(char(10),getdate()-1,120) \
       and A.mdn not in('GUEST'))Z\
       GROUP BY 登入年月\
       ORDER BY 登入年月"
df3 = pd.read_sql(query4, cnxn)


#當月使用項目前10名
query5 = "SELECT TOP 10 CATEGORY,COUNT(MDN) AS 使用門號數\
         FROM(SELECT DISTINCT MDN,CATEGORY\
         FROM PDATA_DW.SVIEW.EVN_GTAPP_LOG  A\
         WHERE OP_DT BETWEEN convert(varchar(4),year(getdate()))+'-'+convert(varchar(2),month(getdate()-1))+'-'+convert(varchar(1),1)\
         and convert(char(10),getdate()-1,120)\
         and A.mdn NOT IN('GUEST')\
         and CATEGORY NOT IN('0 首頁')\
         and CATEGORY NOT IN('1 登入'))X1\
         GROUP BY CATEGORY\
         ORDER BY 使用門號數 DESC"

df4 = pd.read_sql(query5, cnxn).sort_values(['使用門號數'],ascending=True)


#前月使用項目
query6 = "SELECT CATEGORY,COUNT(MDN) AS 使用門號數\
         FROM(SELECT DISTINCT MDN,CATEGORY\
         FROM PDATA_DW.SVIEW.EVN_GTAPP_LOG  A\
         WHERE OP_DT BETWEEN convert(varchar(4),year(getdate()))+'-'+convert(varchar(2),(month(getdate()-1)-1))+'-'+convert(varchar(1),1)\
         and convert(varchar(10),dateadd(dd,-day(dateadd(dd,-day(getdate()),getdate())),getdate()-1),120)\
         and A.mdn NOT IN('GUEST')\
         and CATEGORY NOT IN('0 首頁')\
         and CATEGORY NOT IN('1 登入'))X1\
         GROUP BY CATEGORY\
         ORDER BY 使用門號數 DESC"

df5 = pd.read_sql(query6, cnxn)

#2021年~每月使用項目門號數
query7 ="SELECT CATEGORY,使用年月,COUNT(MDN) AS 使用門號數\
        FROM(SELECT DISTINCT MDN,CATEGORY,CONVERT(VARCHAR(6),OP_DT,112) AS 使用年月\
        FROM PDATA_DW.SVIEW.EVN_GTAPP_LOG  A\
        WHERE OP_DT BETWEEN'2021-01-01' AND dateadd(dd,-day(getdate()),getdate())\
        and A.mdn not in('GUEST')\
        and CATEGORY in('10 新首頁','優惠券成效統計','132 快速繳款','122 門號合約資訊','1311 本期帳單查詢',\
        '13211 信用卡繳款結果','換頁點擊來源統計','36 優惠券','文件下載率','131 帳單查詢'))X1\
        GROUP BY CATEGORY,使用年月\
        ORDER BY 使用年月,使用門號數 DESC"
df6 = pd.read_sql(query7, cnxn)


query8 ="SELECT CONVERT(VARCHAR(6),A.EFF_DT,112) AS 開通年月,POS_TYPE_DESC AS 銷售點,COUNT(A.EFF_DT) AS 開通數\
         FROM ((SVIEW.OFR_SUBSCR A \
         LEFT JOIN SVIEW.OFR_SUBSCR_STAT B ON B.SUBSCR_STAT_CDE = A.SUBSCR_STAT_CDE)\
         LEFT JOIN SVIEW.EVN_SVC_ORD_ITEM D ON D.SVC_ORD_ITEM_CDE = A.DCON_TYPE)\
         LEFT JOIN PDATA_DW.SVIEW.PRT_POS G ON A.POS_ID=g.POS_ID\
         LEFT JOIN PDATA_DW.SVIEW.PRT_POS_TYPE p\
         on substrING(A.pos_id,3,1) = p.POS_TYPE\
         LEFT JOIN PDATA_DW.SVIEW.OFR_APBW_FEE C\
         ON A.RT_PLAN_CDE_FIRST=C.RT_PLAN_CDE AND SVC_CDE = 741\
         LEFT JOIN SVIEW.OFR_PROM F\
         ON A.PROM_CDE=F.PROM_CDE\
         WHERE A.SUBSCR_STAT_CDE NOT IN ('13','39')\
         AND A.EFF_DT BETWEEN '2021-11-01' and dateadd(dd,-day(getdate()),getdate())\
         AND (D.ITEM_DESC NOT LIKE ('%取消') OR D.ITEM_DESC IS NULL)\
         AND (F.PROM_DESC NOT LIKE('%微量上網%') AND F.PROM_DESC NOT LIKE('%MDVPN%')AND F.PROM_DESC NOT LIKE('%企業簡訊%'))\
         AND C.PROD_TYPE IN('4G','5G','5GM')\
         GROUP BY CONVERT(VARCHAR(6),A.EFF_DT,112),POS_TYPE_DESC"

new = pd.read_sql(query8, cnxn)

query9 ="SELECT CONVERT(VARCHAR(6),K1.FIRST_LGN_DT,112) AS 首登年月,\
         CONVERT(VARCHAR(6),A1.EFF_DT,112) AS 開通年月,POS_TYPE_DESC AS 銷售點,\
         COUNT(K1.SUBSCR_ID) AS 首登門號數\
         FROM PDATA_DW.SVIEW.EVN_IVR_TB_APP_MDN_FIRST K1\
         LEFT JOIN SVIEW.OFR_SUBSCR  A1 ON K1.SUBSCR_ID=A1.SUBSCR_ID\
         LEFT JOIN PDATA_DW.SVIEW.PRT_POS G ON A1.POS_ID=g.POS_ID\
         LEFT JOIN PDATA_DW.SVIEW.PRT_POS_TYPE p \
         on substrING(A1.pos_id,3,1) = p.POS_TYPE\
         LEFT JOIN PDATA_DW.SVIEW.OFR_APBW_FEE C\
         ON A1.RT_PLAN_CDE_FIRST=C.RT_PLAN_CDE AND SVC_CDE = 741 \
         LEFT JOIN SVIEW.OFR_PROM  ON A1.PROM_CDE=OFR_PROM.PROM_CDE\
         WHERE K1.FIRST_LGN_DT BETWEEN '2021-11-01' and convert(char(10),getdate()-1,120)\
         AND  CONVERT(VARCHAR(6),K1.FIRST_LGN_DT,112)=CONVERT(VARCHAR(6),A1.EFF_DT,112)\
         GROUP BY CONVERT(VARCHAR(6),K1.FIRST_LGN_DT,112),CONVERT(VARCHAR(6),A1.EFF_DT,112),POS_TYPE_DESC"

newapp = pd.read_sql(query9, cnxn)

new =new[(new['銷售點']!='內部資源通路')&(new['銷售點']!='企業業務')]
newapp =newapp[(newapp['銷售點']!='內部資源通路')&(newapp['銷售點']!='企業業務')]
df7=pd.merge(new, newapp, how="left", on=["開通年月","銷售點"])
df7['當月開通首登APP佔比(%)']=round(((df7['首登門號數'])/(df7['開通數'])*100),1)
df7=df7.sort_values(['開通年月'],ascending=True)

#資料匯出
df.to_csv('df.csv',encoding='mbcs',index=False)
df2.to_csv('df2.csv',encoding='mbcs',index=False)
df3.to_csv('df3.csv',encoding='mbcs',index=False)
df4.to_csv('df4.csv',encoding='mbcs',index=False)
df5.to_csv('df5.csv',encoding='mbcs',index=False)
df6.to_csv('df6.csv',encoding='mbcs',index=False)
df7.to_csv('df7.csv',encoding='mbcs',index=False)